##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 训练检查点

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/guide/checkpoint"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png"> 在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/checkpoint.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png"> 在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/checkpoint.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png"> 在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/checkpoint.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table>

“保存 TensorFlow 模型”这一短语通常表示保存以下两种元素之一：

1. 检查点，或
2. SavedModel。

检查点可以捕获模型使用的所有参数（`tf.Variable` 对象）的确切值。检查点不包含对模型所定义计算的任何描述，因此通常仅在将使用保存参数值的源代码可用时才有用。

另一方面，除了参数值（检查点）之外，SavedModel 格式还包括对模型所定义计算的序列化描述。这种格式的模型独立于创建模型的源代码。因此，它们适合通过 TensorFlow Serving、TensorFlow Lite、TensorFlow.js 或者使用其他编程语言（C、C++、Java、Go、Rust、C# 等 TensorFlow API）编写的程序进行部署。

本文介绍用于编写和读取检查点的 API。

## 设置

In [ ]:
import tensorflow as tf

In [ ]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [ ]:
net = Net()

## 从 `tf.keras` 训练 API 保存

请参阅 [`tf.keras` 保存和恢复指南](./keras/overview.ipynb#save_and_restore)。

`tf.keras.Model.save_weights` 可以保存一个 TensorFlow 检查点。 

In [ ]:
net.save_weights('easy_checkpoint')

## 编写检查点


TensorFlow 模型的持久状态存储在 `tf.Variable` 对象中。这些对象可以直接构造，但通常会通过像 `tf.keras.layers` 或 `tf.keras.Model` 这样的高级 API 创建。

管理变量的最简单方法是将它们附加到 Python 对象，然后引用这些对象。

`tf.train.Checkpoint`、`tf.keras.layers.Layer` 和 `tf.keras.Model` 的子类会自动跟踪分配给其特性的变量。下面的示例构造了一个简单的线性模型，然后编写检查点，其中包含该模型所有变量的值。

您可以使用 `Model.save_weights` 轻松保存模型检查点。

### 手动创建检查点

#### 设置

为了帮助演示 `tf.train.Checkpoint` 的所有功能， 下面定义了一个玩具 (toy) 数据集和优化步骤：

In [ ]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [ ]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### 创建检查点对象

使用 `tf.train.Checkpoint` 对象手动创建一个检查点，其中要检查的对象设置为对象的特性。

`tf.train.CheckpointManager` 也有助于管理多个检查点。

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### 训练模型并为模型设置检查点

以下训练循环可创建模型和优化器的实例，然后将它们收集到 `tf.train.Checkpoint` 对象中。它在每批数据上循环调用训练步骤，并定期将检查点写入磁盘。

In [ ]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [ ]:
train_and_checkpoint(net, manager)

#### 恢复和继续训练

在第一个训练周期结束后，您可以传递一个新的模型和管理器，但在您中断的地方继续训练：

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

`tf.train.CheckpointManager` 对象会删除旧的检查点。上面配置为仅保留最近的三个检查点。

In [ ]:
print(manager.checkpoints)  # List the three remaining checkpoints

这些路径（如 `'./tf_ckpts/ckpt-10'`）不是磁盘上的文件，而是一个 `index` 文件和一个或多个包含变量值的数据文件的前缀。这些前缀被分组到一个单独的 `checkpoint` 文件 (`'./tf_ckpts/checkpoint'`) 中，其中 `CheckpointManager` 保存其状态。

In [ ]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## 加载机制

TensorFlow 通过从加载的对象开始遍历带命名边的有向计算图来将变量与检查点值匹配。边名称通常来自对象中的特性名称，例如 `self.l1 = tf.keras.layers.Dense(5)` 中的 `"l1"`。`tf.train.Checkpoint` 使用其关键字参数名称，如 `tf.train.Checkpoint(step=...)` 中的 `"step"`。

上面示例中的依赖图如下所示：

![Visualization of the dependency graph for the example training loop](https://tensorflow.google.cn/images/guide/whole_checkpoint.svg)

优化器为红色，常规变量为蓝色，优化器插槽变量为橙色。其他节点（例如，代表 `tf.train.Checkpoint` 的节点）为黑色。

插槽变量是优化器状态的一部分，但是是为特定变量创建的。例如，上面的 `'m'` 边对应于动量，Adam 优化器会针对每个变量跟踪该动量。只有在同时保存变量和优化器时，才会将插槽变量保存到检查点中，并因此保存虚线边。

在 `tf.train.Checkpoint` 对象上调用 `restore` 会排队处理请求的恢复，一旦有来自 `Checkpoint` 对象的匹配路径，就会恢复变量值。例如，您可以通过重建一个穿过网络和层到达它的路径来仅从上面定义的模型加载偏差。

In [ ]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # This gets the restored value.

这些新对象的依赖关系计算图是您上面所编写较大检查点的一个小得多的子计算图。它仅包括偏差和 `tf.train.Checkpoint` 用于对检查点进行编号的保存计数器。

![Visualization of a subgraph for the bias variable](https://tensorflow.google.cn/images/guide/partial_checkpoint.svg)

`restore` 返回一个具有可选断言的状态对象。在新的 `Checkpoint` 中创建的所有对象都已恢复，因此 `status.assert_existing_objects_matched` 通过。

In [ ]:
status.assert_existing_objects_matched()

检查点中有许多不匹配的对象，包括层的内核和优化器的变量。`status.assert_consumed()` 仅在检查点和程序完全匹配时通过，并在此处抛出异常。

### 延迟恢复

当输入形状可用时，TensorFlow 中的 `Layer` 对象可能会将变量创建延迟到变量的首次调用。例如，`Dense` 层内核的形状取决于该层的输入和输出形状，因此，作为构造函数参数所需的输出形状没有足够的信息来单独创建变量。由于调用 `Layer` 还会读取变量的值，必须在变量的创建与其首次使用之间进行恢复。

为支持这种习惯用法，`tf.train.Checkpoint` 会对尚不具有匹配变量的恢复进行排队。

In [ ]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### 手动检查检查点

`tf.train.load_checkpoint` 返回一个提供对检查点内容进行较低级别访问权限的 `CheckpointReader`。它包含从每个变量的键到检查点中每个变量的形状和 dtype 的映射。如上面显示的计算图中所示，变量的键是它的对象路径。

注：检查点没有更高级别的结构。它只知道变量的路径和值，而没有 `models`、`layers` 或它们如何连接的概念。

In [ ]:
reader = tf.train.load_checkpoint('./tf_ckpts/')
shape_from_key = reader.get_variable_to_shape_map()
dtype_from_key = reader.get_variable_to_dtype_map()

sorted(shape_from_key.keys())

因此，如果您对 `net.l1.kernel` 的值感兴趣，可以使用以下代码获取该值：

In [ ]:
key = 'net/l1/kernel/.ATTRIBUTES/VARIABLE_VALUE'

print("Shape:", shape_from_key[key])
print("Dtype:", dtype_from_key[key].name)

此外，它还提供了一个 `get_tensor` 方法，允许您检查变量的值：

In [ ]:
reader.get_tensor(key)

### 列表和字典跟踪

对于像 `self.l1 = tf.keras.layers.Dense(5)` 一样的直接特性赋值，将列表和字典分配给特性会跟踪其内容。

In [ ]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

您可能会注意到列表和字典的包装器对象。这些包装器是可设置检查点版本的基础数据结构。就像基于特性的加载一样，这些包装器会在将变量添加到容器后立即恢复它的值。

In [ ]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

相同的跟踪会自动应用于 `tf.keras.Model` 的子类，并且可用于跟踪层列表等用途。

## 总结

TensorFlow 对象提供了一种简单的自动机制来保存和恢复它们所使用变量的值。
